In [1]:
# !pip install jiwer -qq

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from sk import predict
from pathlib import Path
import jiwer
import pandas as pd

# Predict single or multiple file

In [3]:
fn = "/content/test/youtube/0228444ff34081eda587c0ca53712486.wav" # or ["/content/audio1.wav","/content/audio2.wav"]
out = predict(fn)

saved to: /root/.sk/models/conformer_small.onnx
loading model
Total input path: 1
Total audio found(.wav): 1
start prediction


100%|██████████| 1/1 [00:01<00:00,  1.25s/it]


In [4]:
Path(fn).with_suffix(".txt").read_text()

'pada peringkat ini pembukaan sekolah hanya melibatkan'

In [5]:
out

{'entropy': [0.08121918],
 'filenames': [PosixPath('/content/test/youtube/0228444ff34081eda587c0ca53712486.wav')],
 'texts': ['pada peringkat ini pembukaan sekolah hanya melibatkan'],
 'timesteps': [[0]]}

# Predict from single or multiple folder

In [6]:
fn = "/content/test/" # or ["/content/test","/content/data"]
out = predict(fn)

Total input path: 1
Total audio found(.wav): 1700
start prediction


100%|██████████| 1700/1700 [00:43<00:00, 39.48it/s]


In [7]:
out.keys()

dict_keys(['texts', 'filenames', 'entropy', 'timesteps'])

In [8]:
preds = out["texts"]
files = out["filenames"]

In [9]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer'],jiwer.compute_measures([l for l in label],[ii for ii in i])['wer']])

In [10]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer","cer"]
df.head()

,path,label,pred,wer,cer
0,/content/test/youtube/29d818abf5a0403e37cd25f1...,apakah yang akan terjadi jika dia tidak kembal...,apakah yang akan terjadi jika dia tidak kembal...,0.100000,0.060000
1,/content/test/youtube/a3ad200044b4111f31f8f9bc...,aku tahu hari ini hari apa kata dadli bernyany...,aku tahu hari ini hari apa kata dadli bernyany...,0.000000,0.000000
2,/content/test/youtube/79eb99b42174f85011f254fd...,aku dah tak nak jadi ejen,aku tak tak nak jadi je temamawa,0.500000,0.500000
3,/content/test/youtube/cef09fa62bfedf4d94a13b3b...,rumah aku dengan aku kau nak kira,rumah akuenan aku kau nak kira,0.285714,0.074074
4,/content/test/youtube/6e73bafb21ab0d899a5b197f...,karya jk rowling,jk roling,0.666667,0.428571


In [11]:
df["wer"].mean(),df["cer"].mean()

(0.07758634045136972, 0.025199577423303785)